# Disjoint Sets (непересекающиеся множества)

## Непересекающиеся множества на массиве

Иногда требуется структура данных, которая умеет выполнять следующие операции:
* union (O(n))
* find (O(1))

In [1]:
class DisjointSetArray {
private:
    std::vector<int> sets;
public:
    DisjointSetArray(int size);
    virtual ~DisjointSetArray();

    void unionSets(int x, int y);
    int find(int x);

};

In [2]:
DisjointSetArray::DisjointSetArray(int size) {
    sets.resize(size);
    for (int i=0; i<size; i++)
        sets[i] = i;
}

In [3]:
DisjointSetArray::~DisjointSetArray() {
}

In [4]:
void DisjointSetArray::unionSets(int main, int other) {
    if (sets[main] == sets[other])
        return;
    else {
        int t = sets[other];
        for (int i = 0; i < sets.size(); i++)
            if (sets[i] == t)
                sets[i] = sets[main];
    }
}

In [5]:
int DisjointSetArray::find(int x) {
    return sets[x];
}

## Непересекающиеся множества на дереве (с упаковкой в массив)

Каждое множество хранится в виде дерева. Элементы множества хранятся в вершиных дерева. У каждого множества есть его представитель — один из элементов этого множества, он хранится в корне дерева. В каждом узле, кроме корня, хранится ссылка на "родителя".
При объединении двух множеств, корень одного дерева подвешивается к другому (операция union). Таким образом, чтобы определить, в каком множестве находится элемент достаточно пройтись по ссылкам по дереву вверх до корня (операция get).
Без использования дополнительных "улучшений", такое дерево может выродиться в линейный список, где get
будет работать за линейное время, и никакого выигрыша по сравнению с наивными реализациями не будет. Выигрыш в скорости можно получить, используя две эвристики: объединение по рангу (union by rank) и сжатие пути (path compression). 

Объединение по рангу
Эта эвристика аналогична весовой эвристике у связных списков. Идея в том, чтобы при объединении подвешивать дерево с меньшей глубиной к дереву с большей.
Вместо того, чтобы явно хранить высоту дерева, можно хранить его ранг, который по сути является некой верхней оценкой высоты дерева. У дерева, состоящего ровно из одного элемента ранг равен 0
. При объединении дерево с меньшим рангом подвешивается к дереву с большим, и ранг объединенного дерева становится равным большему из этих двух рангов. Если ранги объединяемых деревьев равны, то не важно какое к какому дереву подвешивать, но ранг объединенного дерева следует делать большим на 1
.


Сжатие пути
Эта эвристика несколько модифицирует операцию get. Операция get вызывается для элемента x, проходит через несколько вершин и попадает в корень. Все пройденные в этом процессе вершины принадлежат тому же множеству, что и x. Поэтому мы можем подвесить (изменить ссылки) эти вершины напрямую к корню дерева и, таким образом, уменьшить его высоту. При нерекурсивной реализации операция get становится двухпроходной. 

После всех модификаций получаем сложность:
* union O(logN)
* find O(logN)

In [6]:
class DisjointSetTree {

    std::vector<int> parents;
    std::vector<int> ranks;
public:
    DisjointSetTree(int size);
    virtual ~DisjointSetTree();

    void unionSets(int x, int y);
    int find(int x);

};

In [7]:
DisjointSetTree::DisjointSetTree(int size) {
    ranks.resize(size);
    parents.resize(size);
    for (int i = 0; i < size; i++){
        parents[i] = i;
        ranks[i] = 0;
    }
}

In [8]:
DisjointSetTree::~DisjointSetTree() {
}


In [9]:
void DisjointSetTree::unionSets(int main, int other) {
    main = find(main);
    other = find(other);
    if (main == other)
        return;
    if (ranks[main] == ranks[other])
        ranks[main]++;
    if (ranks[main] < ranks[other])
        parents[main] = other;
    else
        parents[other] = main;
}


In [10]:
int DisjointSetTree::find(int x) {
    if (parents[x] != x)
        parents[x] = find(parents[x]);
    return parents[x];
}